# 1.TRPO代理目标函数的解释
## 1.1 数学符号的介绍
### 1.1.1 数学期望 $\mathbb{E}$
(1)概念：一个随机变量的期望（或者期望值）指的是其概率分布集中趋势的一种度量。

(2)数学原理：
![](./images/数学期望的数学原理.png)

(3)期望的性质：
线性、非负性（如果随机变量非负）、独立性（如果随机变量是互相独立的）。注意后两个性质是有前提条件的。
![](./images/数学期望的性质.png)

(4)数学期望在RL中的应用：
主要用于评估plicy的性能。
![](./images/数学期望在RL中的应用.png)

### 1.1.2 优势函数 Advantage Function
（1）概念：
通过量化在特定状态下采取某个行动的相对质量，与在该状态下所有可能行动的平均预期质量之间的差距，来帮助提高策略优化过程中的稳定性和效率。

（2）数学原理
![](./images/优势函数的数学原理.png)

（3）优势函数的性质
相对比较、泛华性、信号可解释性。
![](./images/数学期望的性质.png)

（4）优势函数的RL应用
![](./images/优势函数的应用_1.png)
![](./images/优势函数的应用_2.png)
![](./images/优势函数的应用_3.png)

### 1.1.3 KL散度 Kullback-Leibler(KL) divergence
（1）概念：KL散度主要用于量化两个概率分布之间的差异。

（2）数学原理：
分为离散分布和连续分布两种情况。
![](./images/KL散度的数学原理.png)

特殊情况：
![](./images/KL散度的特殊计算情况.png)

（3）KL散度的性质
非负性、不对称性、与熵之间的关系、凸性。
![](./images/KL散度的性质.png)

（4）KL散度的应用例子
![](./images/KL散度的应用例子_1.png)

# 2.TRPO目标函数的求解过程
![](./images/TRPO求解的过程概览.jpg)

## 2.1 定义策略优化的目标：
策略优化的目标，就是最大化期望的期望函数的值。
![](./images/TRPO目标函数求解步骤_1.jpg)


## 2.2 定义代理目标：
![](./images/TRPO目标函数求解步骤_2.jpg)

这里之所以叫做代理目标，是因为TRPO并不按照1中的方式，直接优化目标函数，而是对一个新的代理目标进行优化，具体为什么优化代理目标就能得到优化原始目标同样的策略优化效果，具体的数学证明如下。
![](./images/TRPO代理目标函数与原目标函数的等价证明.png)



## 2.3 引入置信域约束：
引入置信域约束的主要原因是在代理目标函数的直接使用朴素梯度下降会导致策略的大幅度更新，导致泛化性差并且难以收敛。而TRPO使用KL散度解决了这个问题，限制了在一次更新中，策略更新的幅度，防止了灾难性的大幅度策略更新。
![](./images/TRPO目标函数求解步骤_3.jpg)



## 2.4 构造约束的优化问题：
![](./images/TRPO目标函数求解步骤_4.jpg)



## 2.5 使用自然梯度下降求解：
由于KL散度的约束导致无法直接求解，因此使用二次估计对KL散度进行近似。近似完成之后就可以使用自然梯度下降法来进行求解。
![](./images/TRPO目标函数求解步骤_5.jpg)

### 2.5.1 二次估计 quadratic approximation
在TRPO中的主要作用就是使用二阶泰勒展开对KL散度约束项进行估计。
![](./images/二次估计KL散度的数学过程.jpg)

### 2.5.2 Fisher information matrix(FIM)
FIM的主要作用是作为KL散度的Hessian(二阶导)近似，使得自然梯度能够高效计算。
![](.//images/FisherInformationMatrix.jpg)

### 2.5.3 Natural Gradient Descent
NGD在TRPO中的主要作用就是求解目标函数的。
![](./images/自然梯度下降法.jpg)

## 2.6 使用共轭梯度进行更新
使用共轭梯度算法原因主要是因为计算自然梯度的方向时，计算开销太大，这里的开销主要指的是计算矩阵的逆的时候，开销很大。
![](./images/TRPO目标函数求解步骤_6.jpg)

### 2.6.1 共轭梯度算法Cojugate gradient algorithm
![](./images/共轭梯度算法原理.jpg)

## 2.7 使用线搜索来进行最终的更新
使用线搜索的主要原因是为找到最佳的step size $\alpha$ <br>
![](./images/TRPO目标函数求解步骤_7.jpg)

### 2.7.1 Line Search
线搜索是一种优化方法，通常用于在给定方向下移动时找到最佳的step size。通常与基于梯度的算法结合，比如梯度下降，共轭梯度等。
![](./images/线搜索的数学原理.jpg)

# 3.裁剪代理目标Clipped Surrogate Objective
## 3.1 动机
TRPO的代理目标是$L^{CPI}$，CPI指的是传统的策略迭代，存在的问题是，如果在没有KL散度的约束条件下，直接优化CPI会导致policy的过度更新。因此PPO修改了目标，该目标中惩罚了概率比的远离1的时候的更新（意思就是policy更新的幅度）。

## 3.2 修改后的目标
![](./images/clipped代理目标.jpg)

1> 其中第一项其实就是原始的CPI的目标，第二项限制了概率比在一个由$\epsilon$限制的范围内，一般取值是0.2。<br>
2> 第二项对概率比的裁剪，会是的概率比的移动会被限制在这个区间内。<br>
3> $L^{CLIP}$实际上是$L^{CPI}$的一个下界。通过这种方式，会忽略使得policy变得更好的概率比变化，而会纳入使得policy变差的变化<br>

## 3.3  裁剪代理目标的性质
（1）概率比是在$1-\epsilon$被裁剪还是$1 + \epsilon$，是取决于优势函数的值A是大于0还是小于0的。<br>
A > 0的时候，在$1 + \epsilon$被裁剪；<br>
A < 0的时候，在$1-\epsilon$被裁剪。<br>
![](./images/clipped代理目标的性质_1.jpg)<br>

（2）$L^{CLIP}$实际上是$L^{CPI}$的一个下界,当policy更新幅度太大的时候，会被惩罚。
![](./images/clipped代理目标的性质_2.jpg)<br>


# 4.自适应KL惩罚系数Adaptive KL Penalty Coefficient
## 4.1 动机
为了提出一个clipped surrogate objective的替代选择，或者是与之结合使用的选择：那就是在KL散度上添加惩罚系数。

## 4.2 实际的效果：
这种在KL散度上添加惩罚系数的效果比裁剪代理目标的效果差，但是在这儿作为baseline提出来

## 4.3 算法的步骤
(1)使用随机梯度下降算法优化$L^{KLPEN}$；<br>
(2) 计算KL散度的期望 d，并且根据 d 的大小来更新惩罚系数 $\beta$。<br>
(3)算法流程中参数更新的解释：更新d的判断条件中的常数是超参数，但是算法对他们的值并不敏感。而且惩罚系数 $\beta$的初始值也不重要，因为过程中它很快就会被调整。
![](./images/自适应KL惩罚系数.jpg)


# 5.算法
## 5.1 最终的目标函数
目标函数组成部分的介绍：<br>
（1）第一项就是之前介绍的裁剪代理目标；<br>
（2）第二项是价值函数的loss（也叫做critic loss），是均方误差，用于降低梯度更新过程中的variance.<br>
（3）第三项是熵增（Entropy Loss），主要的作用是防止policy陷入局部最优。<br>
![](./images/PPO最终的目标函数.jpg)<br>
![](./images/PPO最终的目标函数_价值函数loss.jpg)<br>
![](./images/PPO最终的目标函数_熵loss.jpg)<br>

# 补充知识：variance 和 bias的区别与联系
![](./images/偏差和方差的联系.jpg)
![](./images/偏差和方差的区别.jpg)

## 5.2 梯度更新的实现
### 5.2.1 PPO的算法流程
在一次迭代过程中：<br>
(1)对N个并行的actors，分别收集T个时间步的结果；<br>
(2)在NT个时间步的数据上，构建 surrogate loss，并且使用SGD或者Adam优化器进行优化K个epochs。<br>
![](./images/PPO算法流程.jpg)<br>

### 5.2.2 广义优势估计
在强化学习中，策略梯度方法需要一种方式来估计优势函数（advantage function）和回报（Return），以指导策略更新和价值函数优化。PPO就是使用了GAE来对优势函数进行平滑计算，兼顾短期和长期的奖励信号。具体来讲，就是在构建surrogate loss的过程中，会需要得到$L^{CLIP}$中优势函数的advantage estimator。在PPO中是使用truncated version of generalized advantage estimation广义优势估计（GAE）得到的。具体的数学表达式如下：<br>
这里公式11的最后一项的上标错了，正确的应该是T-t-1。可以参考https://www.zhihu.com/question/5395314304。
![](./images/公式11的推导.jpg)<br>
![](./images/PPO_GAE.jpg)<br>

![](./images/PPO_GAE_数学原理.jpg)<br>